
# データ構造化事例　＃２
## 走査電子顕微鏡（SEM）のデータ構造化 



**対応機種** ：　 日立ハイテク　TM3000, TM4000シリーズ  
**rawデータ**：　txt (テキスト形式)  
**スクリプトの内容**：  
本ドキュメントは卓上電子顕微鏡（低真空SEM）のTM3000, TM4000シリーズについて，撮影した画像を登録することのほかに，撮影情報をリスト化するデータ構造化のスクリプトに関する動作方法について記述します．

![image](https://user-images.githubusercontent.com/38028745/133568158-b10fe1c0-8024-434e-8bd6-23d71cfb6ffb.png)

## サンプルデータを読み込みましょう

In [1]:
!git clone https://github.com/ARIM-Japan/Training_Program_2.git
%cd Training_Program_2

Cloning into 'Training_Program_2'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (138/138), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 138 (delta 71), reused 58 (delta 20), pack-reused 0
Receiving objects: 100% (138/138), 2.89 MiB | 27.40 MiB/s, done.
Resolving deltas: 100% (71/71), done.
/content/Training_Program_2


これで準備が整いました．

<hr>

## 1. ライブラリーを読み込みましょう
最初にコードを走らせるのに必要なライブラリーを読み込みます．ライブラリとしては以下を使っています．
 * 標準ライブラリー: ``sys``,``glob``, ``os``, ``io``, ``re``, ``csv``,``chardet``
 * 数値処理用: ``pandas``
 
 目新しいライブラリーとしては``chardet``でしょうか．これはファイルの読み込みにおいて，文字コードの判定に使うライブラリーです． ファイルの中が英語，日本語によってエンコーディングをutf-8とするか，それともshift-jisとするかの判定を行う場面が生じます．このときに``chardet``のライブラリーが役に立ちます． 
 https://pypi.org/project/chardet/

In [2]:
# ファイル操作用
import sys
import glob
import os
import io
import re
import csv

# 数値処理用
import pandas as pd
from chardet import detect

また，出力結果を「output」フォルダーに保存するため，そのフォルダーの作成をします．

In [3]:
output_folder = 'output'
os.makedirs(output_folder,exist_ok = True)

## 2. ファイルの拡張子を調べます
データ構造化の最初のステップとして，データ構造化するファイルを特定する必要があることは＃１で述べた通りですが，ここでも特定は**拡張子**で判定します． 

下記の``read_files``関数は＃１よりもシンプルにしていますが，入力ファイルが置いてある「data」フォルダーにある特定の拡張子のファイルをすべてリスト化する考え方は同じくしています．特にSEMでは倍率や場所を変えて複数の撮影を行うため，複数のファイルを事前に判定しておく必要があります．

もしエラーがでたら「data」フォルダーを作成し，その中に.txtファイルを配置してみてください．

In [4]:
# 実行処理#rawデータの読み込み
def read_files(extension):
    """
    概要: 特定の拡張子のついたファイルをすべて読み込む
    @param extension: 読み込む拡張子
    @return ソートされたファイル名群
    """

    #入力データのフォルダーを"data"とする
    data_folder = 'data/*'
    path = data_folder + extension
    
    # 連続撮影されたファイル名の確認
    files = glob.glob(path)
    
    input_files = sorted(files)
    print (input_files)
    
    return input_files

サンプルファイルではSEMで撮影された三枚の写真があります．それらの撮影情報ファイルである.txtのファイルを指定してみましょう．

In [5]:
extension = '.txt'
files = read_files(extension)

['data/sample-１_1200倍(x1.2k).txt', 'data/sample-１_200倍(x200).txt', 'data/sample-１_600倍(x600).txt']


上記のように複数撮影されたリスト配列の結果がソートされて戻ります． 

## 3. メタデータのデータ構造化　（辞書および翻訳対応の作成）

撮影情報ファイルには多くのメタデータが含まれています．このままでは使いにくいですよね．      
その中から加速電圧や倍率など論文などに記載が必要なメタデータのみを抽出してリスト化すると，SEMの写真管理でも役に立ちます．  

ここでは，その作業のはじめとして，まずは必要なメタデータを外部辞書ファイル（``KEYPARAFILE.csv``）として作成しておき，それをpythonの中ではdictionaly構造でkeyとして保持させます．

後の＃５では，この辞書とマッチしたメタデータのvalueを返すようにしますが，そのための仕掛けとなります．

ここではkeyを英語から日本語，もしくは日本語から英語へと変換させることができます．また，メーカー固有の出力を分かりやすい世用語に置き換える「翻訳」をさせることも可能です．

In [6]:
def get_keypara(_fn):

    """
    概要: KEYPARAFILEのcsvを読み込みメタデータを翻訳する
    @param _fn: templateファイル
    @return：　変換した語彙の辞書を辞書型として
    """
    
    _dict = dict()

    try:
        with open(_fn, mode='r') as f:
            line = f.readline()
            
            while line:
                str = line.strip().split(',')
                key = str[0].strip()
                value = str[1].strip()
                _dict[key] = value
                line = f.readline()
    except:
        print('get_keypara NG', file=sys.stderr)

    return _dict

予めSEMの必要としたパラメータをcsvとしてまとめたキーパラメータのリスト``keypara.csv``を定義しておきます．そして，`` get_keypara``関数で次のように走らせます．

In [7]:
KEYPARAFILE = 'keypara.csv'
keypara = get_keypara(KEYPARAFILE)
keypara

{'AcceleratingVoltage': 'Accelerating Voltage',
 'DataNumber': 'Comment and Number',
 'DataSize': 'Data Size',
 'Date': 'Measurement Date',
 'ImageName': 'File Name',
 'InstructName': 'Instrument Model',
 'Magnification': 'Magnification',
 'PixelSize': 'Pixel Size',
 'SerialNumber': 'Instrument S/N',
 'SignalName': 'Signal Name',
 'Time': 'Measurement Time',
 'WorkingDistance': 'Working Distance'}

上記のようにkeypara.csvに指定してあるパラメータが辞書化されて取り出されています．

## 4. ファイルの記述形式をcsv型に変える
テキスト型で出力される中でも表記には流儀があります．日立ハイテクのSEMの出力のフォーマットはINIファイル形式の表記となっています．

https://ja.wikipedia.org/wiki/INI%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB

INIファイルにおいてはメタデータのキーと値が”＝”で結ばれていますが，ここでは”＝”から”，”に置き換えてcsv化させる前処理を行います．

次のコードでは読み込みが二段階で行われているように見えますが，最初の``open``関数での読み込みは文字コード判定のために行っています．

あえてSEMの場合にこの処理を加えているのは，SEMの利用形態においては日本語文字でファイル名を保存したり，またファイルの中身においても日本語で記入する割合が高い利用形態が調査でわかりました．  

半角英数字のみであればデフォルトのutf-8として，この処理は不要ですが，日本語が交じるとデフォルトのままでは文字化けが発生するため，この事前の判定を必要とします．  

もし，機器の運用に置いて日本語を使用しないことがわかっていれば，この事前処理は不要です．

In [8]:
def ini_to_csv(_fn):
    """
    概要: ini形式のtextファイルをcsv型へ変換する
    @param _fn: rawデータのファイル
    @return：　csv型をtxtとして戻す
    """
    
    csv = ""
    
    # encodingの判定
    with open(_fn, 'rb') as f:
        b = f.read()
        enc = detect(b)
        encd = enc['encoding']

    with open(_fn, mode='r', encoding=encd) as f:
        line = f.readline()
        
        while line:
            if not re.match(u"\[", line):
                str = line.rstrip(os.linesep).replace('=', ',', 1)
                csv += str
                csv += "\n"
            line = f.readline()
            
    return csv

ではサンプルファイルの一つ「sample-１_1200倍(x1.2k).txt」で走らせて確認しましょう．

In [9]:
filename = "data/sample-１_1200倍(x1.2k).txt"
txt = ini_to_csv(filename)
txt

'InstructName,TM4000\nSemVersion,01-08\nSDMVersion,\nSerialNumber,2170A6-05\nDataNumber,TM4000 0001\nSampleName,\nFormat,tif\nImageName,タンポポ花粉-１_1200倍(x1.2k).tif\nDirectory,D:\\SemImage\\lixianglan\\\nDate,2021/06/23\nTime,16:43:33\nMedia,HD[Data]\nDataSize,1280x960\nPixelSize,82.68229\nSignalName,BSE\nDisplaySignalName,BSE\nSEDetSetting,\nAcceleratingVoltage,15000 Volt\nDecelerationVoltage,0 Volt\nDecelerationMode,\nMagnification,1200\nWorkingDistance,7920.5 um\nEmissionCurrent,46900 nA\nPhotoSize,1000\nMagnificationDisplay,0\nVacuum,50\nMicronMarker,40000\nSubMagnification,0\nSubSignalName,\nSpecimenBias,0 V\nCondencer1,9230\nScanSpeed,Capture_Slow(80)\nCaptureSpeed_Integration,\nCalibrationScanSpeed,24\nImgEnhance,0\nColorMode,Grayscale\nColorPalette,\nScreenMode,Full Screen\nComment,\nKeyWord1,\nKeyWord2,\nCondition,Vacc=15.0kV Mag=x1.20k WD=7.9mm LensMode=3\nDataDisplayCombine,1\nStageType,0\nStagePositionX,17500000\nStagePositionY,17500000\nStagePositionR,0.000\nStagePositionZ,0\

csvであるはずが，文字の羅列のようにみえますね．実際，プログラムの中ではcsvはこのような形で格納されています．

特に文字化けのようにみえる「\n」（出力は円記号）がリターンの意味です．このようにして通常の表示では改行されて表示されますが，ここでは文字列が連続的に表示されます．

## 5. 撮影情報メタリストの作成
最後の処理となります．  
ここでは＃４の出力の中から＃３で指定したメタデータのみを抽出します．そして，複数ファイルから同じメタデータのみを抽出し，それらを表組化する処理を行います．

In [10]:
def make_metatable (files,keypara):
    
    # 撮影メタデータの抽出
    n = 1
    for fn in files:
        txt = ""
        txt = ini_to_csv(fn)
        
        tmp = pd.read_csv(io.StringIO(txt), header=None,
                          names=['key', 'value%s' % n])
        if n == 1:
            out = tmp
        else:
            out = pd.merge(out, tmp, on='key', how='outer')
        n += 1
        
    out = out.set_index('key')
    out = out.transpose()
    out = out[list(keypara.keys())]
    d = dict(zip(list(keypara.keys()), list(keypara.values())))
    out = out.rename(columns=d)

    # 単位削除
    for index, row in out.iterrows():
        for k in ['Accelerating Voltage', 'Working Distance']:
            v = row[k]
            if v == v:
                v = v.split(' ')
                if len(v) > 1:
                    row[k] = v[0]

    # リストの出力
    out.to_csv('output/' + "metalist.csv", index = None,
               quoting=csv.QUOTE_NONNUMERIC, encoding='utf_8_sig')
    
    return out

では，結果を見てみることにしましょう．
一連のファイル群は＃２のfilesに入っています．それをそのまま流し込みます．また，keyparaは＃３でセレクトしたものです．それらを``make_metatable``関数に入れます．

In [11]:
files = files
keypara = keypara
out = make_metatable (files,keypara)
out

key,Instrument Model,Instrument S/N,Comment and Number,File Name,Measurement Date,Measurement Time,Data Size,Pixel Size,Signal Name,Accelerating Voltage,Magnification,Working Distance
value1,TM4000,2170A6-05,TM4000 0001,タンポポ花粉-１_1200倍(x1.2k).tif,2021/06/23,16:43:33,1280x960,82.68229,BSE,15000,1200,7920.5
value2,TM4000,2170A6-05,TM4000 0001,タンポポ花粉-１_200倍(x200).tif,2021/06/23,16:38:33,1280x960,496.0938,BSE,15000,200,7706.35
value3,TM4000,2170A6-05,TM4000 0001,タンポポ花粉-１_600倍(x600).tif,2021/06/23,16:41:13,1280x960,165.3646,BSE,15000,600,7892.3


必要とするメタデータが抽出されて表組みになったのがわかります．撮影情報として最低限に必要な項目のみが綺麗にまとめられると，撮影したファイル名から撮影目的も一目瞭然となります．

あわせて，この組み合わせが「output」フォルダに.csvとして出力されています．

## ６. 一本化する
上記の一連の処理を一本化することをします．下記のように一本化したコードを``main``関数として定義しておきます．処理の順番を並べるだけです．


In [12]:
def main():

    output_folder = 'output'
    os.makedirs(output_folder,exist_ok = True)
    
    extension = '.txt'
    files = read_files(extension)
    
    KEYPARAFILE = 'keypara.csv'
    keypara = get_keypara(KEYPARAFILE)
    
    
    out = make_metatable(files,keypara)
    
    print (out)

その``main``関数を下記のようにして走らせてみましょう．一連の処理が一貫で行われます．

In [13]:
main()


['data/sample-１_1200倍(x1.2k).txt', 'data/sample-１_200倍(x200).txt', 'data/sample-１_600倍(x600).txt']
key    Instrument Model Instrument S/N  ... Magnification Working Distance
value1           TM4000      2170A6-05  ...          1200           7920.5
value2           TM4000      2170A6-05  ...           200          7706.35
value3           TM4000      2170A6-05  ...           600           7892.3

[3 rows x 12 columns]


では出力されたCSVを取り出して見てみましょう．PCをお使いの方々はoutput_folderというフォルダーを見れば，グラフとCSVが出力されています．Google Colaboratoryをお使いの方々は，まず下のコマンドコードを実行してください．

In [14]:
!pwd
!ls

/content/Training_Program_2
data  keypara.csv  output  README.md  Training_2.ipynb


現在プログラムを実行しているフォルダーがTraining_Program_1というところで，そこにdataフォルダーやoutputフォルダー，READMEやこのプログラムの書かれたノートブックのファイルであるTraining_1.ipynbもあります．では、次のコマンドコードでoutputフォルダーに移動してみましょう．

In [15]:
%cd output/
!ls

/content/Training_Program_2/output
metalist.csv


metalist.csvというCSVファイルが出てきました．これを以下のプログラムコードを実行して取り出して見ましょう．

In [16]:
from google.colab import files

files.download("metalist.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CSVファイルが無事にダウンロード出来ましたでしょうか．

## おわりに
このSEMは最も基本形のシンプルなタイプのメタデータを抽出するデータ構造化の事例となります．基本的なポイントとなるのは次の処理項目です．

* データ構造化したいファイルを指定して読み込む
* 抽出するメタデータを外部辞書ファイルとして指定して，それを読み込む
* 実験ファイルの中でマッチングするメタデータを抽出する．

あとは，お好みに応じて``KEYPARA``ファイルの中身を増やしたり減らしたりすることで，ユーザーの利用形態にあわせて調整することができます．

また，SEMの場合には撮影した目的に応じた判定をしているはずです．（例えば欠陥があった，なかったなど）　そのようなラベルリストは別に用意しておくと，機械学習に供することができるようになります．